In [45]:
import tensorflow as tf
from tensorflow.keras.layers import Concatenate

In [46]:
from tensorflow.keras.models import load_model

In [47]:
#getting the test_dataset and batching it
import os
Root_directory = '/home/sanven0212/PycharmProjects/Eye_Disease_Classification/Dataset/eye_disease/split1'
test_dir = os.path.join(Root_directory, 'test')

In [48]:
IMG_SHAPE = 224
BATCH_SIZE = 32

In [49]:
test_data = tf.keras.preprocessing.image_dataset_from_directory(
                                                                 directory = test_dir, 
                                                                 batch_size = BATCH_SIZE,  
                                                                 seed = 123, 
                                                                 shuffle = False, 
                                                                 image_size = (IMG_SHAPE, IMG_SHAPE), 
                                                                 label_mode = 'categorical')

Found 845 files belonging to 4 classes.


In [50]:
def preprocess_img(image, label, img_size = IMG_SHAPE):
    image = tf.image.resize(image, [img_size, img_size])
    image = tf.cast(image, tf.float32)
    return image, label

In [51]:
test_data = test_data.map(map_func = preprocess_img, num_parallel_calls = tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)
test_data_DenseNet_121 = test_data.map(lambda x, y: (tf.keras.applications.densenet.preprocess_input(x), y), num_parallel_calls = tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)

In [52]:
Root_model_path = '/home/sanven0212/PycharmProjects/Eye_Disease_Classification/DL_Models'
ensemble_model = [ ]
list_of_models = os.listdir(Root_model_path)
list_of_models

['VGG_19_89.35',
 'DenseNet_201_91.01',
 'DenseNet_169_90.41',
 'VGG_16_90.18',
 'DenseNet_121_88.64']

In [53]:
for models in os.listdir(Root_model_path):
    DLModel = load_model(os.path.join(Root_model_path, models))
    ensemble_model.append(DLModel)

In [54]:
Model_1_preds = tf.argmax(ensemble_model[0].predict(test_data), axis = 1)
Model_2_preds = tf.argmax(ensemble_model[1].predict(test_data), axis = 1)
Model_3_preds = tf.argmax(ensemble_model[2].predict(test_data), axis = 1)
Model_4_preds = tf.argmax(ensemble_model[3].predict(test_data), axis = 1)
Model_5_preds = tf.argmax(ensemble_model[4].predict(test_data_DenseNet_121), axis = 1)

27/27 [==============================] - 2s 52ms/step


In [55]:
from sklearn.metrics import accuracy_score

In [56]:
#Getting the True Labels
True_labels = []
for images, labels in test_data.unbatch():
    True_labels.append(labels.numpy().argmax())

In [57]:
True_labels

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [59]:
import pandas as pd
Preds_Df = pd.DataFrame({list_of_models[0]: Model_1_preds, list_of_models[1]: Model_2_preds, list_of_models[2]: Model_3_preds, list_of_models[3]: Model_4_preds, list_of_models[4]: Model_5_preds}) 
Preds_Df.head(10)

,VGG_19_89.35,DenseNet_201_91.01,DenseNet_169_90.41,VGG_16_90.18,DenseNet_121_88.64
0,0,0,0,0,0
1,0,0,0,0,0
2,0,3,3,3,3
3,0,0,0,0,0
4,0,3,3,3,3
5,3,0,0,0,0
6,3,0,3,0,2
7,0,0,0,0,0
8,0,3,3,3,0
9,0,0,0,0,0


In [60]:
Preds_Df['Max_Count'] = Preds_Df.apply(lambda x: x.mode().iloc[0], axis = 1) 

In [61]:
Preds_Df

,VGG_19_89.35,DenseNet_201_91.01,DenseNet_169_90.41,VGG_16_90.18,DenseNet_121_88.64,Max_Count
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,3,3,3,3,3
3,0,0,0,0,0,0
4,0,3,3,3,3,3
...,...,...,...,...,...,...
840,3,3,3,3,3,3
841,3,3,3,3,3,3
842,3,3,3,3,3,3
843,3,3,3,3,3,3


In [62]:
accuracy = accuracy_score(True_labels, Preds_Df['Max_Count'])
accuracy

0.9218934911242603

In [63]:
preprocess_input_nasnet = tf.keras.applications.nasnet.preprocess_input
preprocess_input_resnet = tf.keras.applications.resnet.preprocess_input
preprocess_input_xception = tf.keras.applications.xception.preprocess_input
preprocess_input_effb7 = tf.keras.applications.efficientnet.preprocess_input
preprocess_input_inceptionresnetv2 = tf.keras.applications.inception_resnet_v2.preprocess_input
preprocess_input_mobilenetv2 = tf.keras.applications.mobilenet_v2.preprocess_input
preprocess_input_mobilenetv3L = tf.keras.applications.mobilenet_v3.preprocess_input
preprocess_input_resnetrs = tf.keras.applications.resnet_rs.preprocess_input

fundus_test = "/home/sanven0212/PycharmProjects/Eye_Disease_Classification/Dataset/eye_disease/split1/test"


BATCH_SIZE = 32
IMG_HEIGHT, IMG_WIDTH = 224,224
test_dataset_new = tf.keras.preprocessing.image_dataset_from_directory(
    fundus_test,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=False,
    seed=123,
)

IMG_HEIGHT_x, IMG_WIDTH_x = 299,299
test_dataset_xception = tf.keras.preprocessing.image_dataset_from_directory(
    fundus_test,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT_x, IMG_WIDTH_x),
    shuffle=False,
    seed=123,
)
IMG_HEIGHT_mobilenet, IMG_WIDTH_mobilenet = 512,512
test_dataset_mobilenet = tf.keras.preprocessing.image_dataset_from_directory(
    fundus_test,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT_mobilenet, IMG_WIDTH_mobilenet),
    shuffle=False,
    seed=123,
)

Found 845 files belonging to 4 classes.
Found 845 files belonging to 4 classes.
Found 845 files belonging to 4 classes.


In [65]:
test_datagen_nasnet = test_dataset_new.map(lambda x, y: (preprocess_input_nasnet(x), y))
test_datagen_resnet = test_dataset_new.map(lambda x, y: (preprocess_input_resnet(x), y))
test_datagen_xception = test_dataset_xception.map(lambda x, y: (preprocess_input_xception(x), y))
test_datagen_effb7 = test_dataset_new.map(lambda x, y: (preprocess_input_effb7(x), y))
test_datagen_inceptionresnetv2 = test_dataset_xception.map(lambda x, y: (preprocess_input_inceptionresnetv2(x), y))
test_datagen_mobilenetv2 = test_dataset_mobilenet.map(lambda x, y: (preprocess_input_mobilenetv2(x), y))
test_datagen_mobilenetv3L = test_dataset_new.map(lambda x, y: (preprocess_input_mobilenetv3L(x), y))
test_datagen_resnetrs = test_dataset_new.map(lambda x, y: (preprocess_input_resnetrs(x), y))

In [66]:
ensemble_model_2 = []
New_Models_Path = '/home/sanven0212/PycharmProjects/Eye_Disease_Classification/DL_Models/CompletedModels_9/CompletedModels'
for models in os.listdir(New_Models_Path):
    DLModel = load_model(os.path.join(New_Models_Path, models))
    ensemble_model_2.append(DLModel)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 4070 Laptop GPU, compute capability 8.9


2024-07-29 18:14:46.788598: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


In [67]:
ensemble_model_2

In [68]:
List_of_Models_2 = os.listdir(New_Models_Path)
List_of_Models_2

['Nasnet_88.99',
 'EfficientB7_90.49',
 'Mobilenetv2_88',
 'MobilenetV3L_87.7',
 'Resnet_rs_101_87',
 'Xception_new_90.41',
 'Resnet_152_88',
 'InceptionResnetV2_90.65',
 'Resnet_50_90']

In [74]:
Model_6_preds = tf.argmax(ensemble_model_2[0].predict(test_datagen_nasnet), axis = 1)
Model_7_preds = tf.argmax(ensemble_model_2[1].predict(test_datagen_effb7), axis = 1)
Model_8_preds = tf.argmax(ensemble_model_2[2].predict(test_datagen_mobilenetv2), axis = 1)
Model_9_preds = tf.argmax(ensemble_model_2[3].predict(test_datagen_mobilenetv3L), axis = 1)
Model_10_preds = tf.argmax(ensemble_model_2[4].predict(test_datagen_resnetrs), axis = 1)
Model_11_preds = tf.argmax(ensemble_model_2[5].predict(test_datagen_xception), axis = 1)
Model_12_preds = tf.argmax(ensemble_model_2[6].predict(test_datagen_resnet), axis = 1)
Model_13_preds = tf.argmax(ensemble_model_2[7].predict(test_datagen_inceptionresnetv2), axis = 1)
Model_14_preds = tf.argmax(ensemble_model_2[8].predict(test_datagen_resnet), axis = 1)

27/27 [==============================] - 1s 28ms/step


In [75]:
Preds_Df_new = pd.DataFrame({list_of_models[0]: Model_1_preds, list_of_models[1]: Model_2_preds, list_of_models[2]: Model_3_preds, list_of_models[3]: Model_4_preds, list_of_models[4]: Model_5_preds, List_of_Models_2[0]: Model_6_preds, 
                             List_of_Models_2[1]: Model_7_preds,
                             List_of_Models_2[2]: Model_8_preds,
                             List_of_Models_2[3]: Model_9_preds,
                             List_of_Models_2[4]: Model_10_preds,
                             List_of_Models_2[5]: Model_11_preds,
                             List_of_Models_2[6]: Model_12_preds,
                             List_of_Models_2[7]: Model_13_preds,
                             List_of_Models_2[8]: Model_14_preds}) 
Preds_Df_new.head(10)

,VGG_19_89.35,DenseNet_201_91.01,DenseNet_169_90.41,VGG_16_90.18,DenseNet_121_88.64,Nasnet_88.99,EfficientB7_90.49,Mobilenetv2_88,MobilenetV3L_87.7,Resnet_rs_101_87,Xception_new_90.41,Resnet_152_88,InceptionResnetV2_90.65,Resnet_50_90
0,0,0,0,0,0,3,0,0,0,0,0,0,3,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,3,3,3,3,3,3,3,3,3,3,3,3,3
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,3,3,3,3,3,3,3,3,0,3,0,2,3
5,3,0,0,0,0,0,2,3,0,2,3,2,2,2
6,3,0,3,0,2,3,3,3,3,3,0,3,3,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,3,3,3,0,3,0,3,0,0,3,3,3,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [76]:
Preds_Df_new['Max_Count'] = Preds_Df_new.apply(lambda x: x.mode().iloc[0], axis = 1)
Preds_Df_new

,VGG_19_89.35,DenseNet_201_91.01,DenseNet_169_90.41,VGG_16_90.18,DenseNet_121_88.64,Nasnet_88.99,EfficientB7_90.49,Mobilenetv2_88,MobilenetV3L_87.7,Resnet_rs_101_87,Xception_new_90.41,Resnet_152_88,InceptionResnetV2_90.65,Resnet_50_90,Max_Count
0,0,0,0,0,0,3,0,0,0,0,0,0,3,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,3,3,3,3,3,3,3,3,3,3,3,3,3,3
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,3,3,3,3,3,3,3,3,0,3,0,2,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
840,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
841,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
842,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
843,3,3,3,3,3,2,3,3,3,3,3,3,3,3,3


In [77]:
accuracy_score(True_labels, Preds_Df_new['Max_Count'])

0.9242603550295858